# Round 1

In [65]:
PATH = 'NHIS_BDC_2023/Round1/'

In [3]:
!pip install py7zr
!pip install newspaper3k
!pip install yfinance


INFO: pip is looking at multiple versions of py7zr to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.6/138.6 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.6/93.6 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.7/66.7 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 34.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 8.8 MB/s eta 0:00:00:00:01:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 kB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.1/81.1 kB 8.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 25.4 MB/s eta 0:00:0000:

In [4]:
import torch
import py7zr
import numpy as np
import pandas as pd
from newspaper import Article

import os
import time
import re
import warnings

import yfinance as yf

warnings.filterwarnings("ignore")

# 1. RSS IFO 전처리 및 본문 크롤링
- 제공받은 nasdaq_rss_ifo의 데이터 중 본 분석에 맞도록 전처리 및 본문 크롤링을 실행함
- nasdaq_df_wo_text.csv

## 행 필터링

In [7]:
# 모든 CSV 파일을 불러와서 하나의 데이터프레임으로 합치기
dfs = []
for filename in os.listdir(PATH):
    if filename.startswith('NASDAQ_RSS_IFO'):
        print(filename)
        month_df = pd.read_csv(os.path.join(PATH, filename), encoding='latin1')
        dfs.append(month_df)

# 모든 데이터프레임을 하나로 합치기
df = pd.concat(dfs, ignore_index=True)

# 중복 제거
df = df.drop_duplicates(keep='first')

NASDAQ_RSS_IFO_202305.csv
NASDAQ_RSS_IFO_202306.csv
NASDAQ_RSS_IFO_202307.csv
NASDAQ_RSS_IFO_202308.csv
NASDAQ_RSS_IFO_202301.csv
NASDAQ_RSS_IFO_202302.csv
NASDAQ_RSS_IFO_202303.csv
NASDAQ_RSS_IFO_202304.csv


In [8]:
df.shape # (146914, 8)

(146914, 8)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 146914 entries, 0 to 2295463
Data columns (total 8 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   rgs_dt              146914 non-null  int64 
 1   tck_iem_cd          146914 non-null  object
 2   til_ifo             146914 non-null  object
 3   ctgy_cfc_ifo        146914 non-null  object
 4   mdi_ifo             146914 non-null  object
 5   news_smy_ifo        146914 non-null  object
 6   rld_ose_iem_tck_cd  146914 non-null  object
 7   url_ifo             146914 non-null  object
dtypes: int64(1), object(7)
memory usage: 10.1+ MB


In [10]:
#url 기준 중복 행 제거
df['url_base'] = df['url_ifo'].apply(lambda x: re.sub(r'-0$', '', x))
print(df.shape)
df.drop_duplicates(subset=['url_ifo'], inplace=True)
print(df.shape)
df = df[~((df.duplicated(subset='url_base', keep=False)) & (df['url_ifo'].str.endswith('-0')))] #-0 앞까지 중복되면서, -0으로 끝나는 url이 들어간 행 모두 제거
print(df.shape)
df.drop(columns=['url_base'], inplace=True)
print(df.shape)

(146914, 9)
(93241, 9)
(89022, 9)
(89022, 8)


In [11]:
#url이 "_"인 기사를 삭제함
#pre-market, after-hours 관련 기사는 단순히 주가를 나열하는 기사이므로 삭제함

df = df[~(df['url_ifo']=="_")]
print(df.shape)
df = df[~df['ctgy_cfc_ifo'].str.contains("Pre-Market|After-Hours")]
print(df.shape)
df = df[~df['til_ifo'].apply(lambda x: 'Pre-Market' in x)]
print(df.shape)
df = df[~df['til_ifo'].apply(lambda x: 'After-Hours' in x)]
print(df.shape)

(89021, 8)
(88769, 8)
(88676, 8)
(88602, 8)


In [12]:
df.shape

(88602, 8)

### all_tck_iem_cd 열 생성

- 본래 tck_iem_cd 열의 주식을 대상으로 분석을 하려고 했으나, 해당 열이 정확하지 않다고 판단함.
- 예를 들어, 'AAPL'은 해당 열에 7, 8월에만 등장하는데, 이는 상식적이지 않음
- rld_ose_iem_tck_cd 열에는 'AAPL'이 지속적으로 등장하는 것으로 보아, 두 열의 티커코드를 합친 all_tck_iem_cd 열을 생성하여 이를 활용한 분석을 시행하는 게 적절하다고 판단함

In [13]:
print('AAPL이 처음 등장하는 날짜: ', min(df[df['tck_iem_cd']=='AAPL']['rgs_dt']))
print('AAPL이 마지막으로 등장하는 날짜: ', max(df[df['tck_iem_cd']=='AAPL']['rgs_dt']))

AAPL이 처음 등장하는 날짜:  20230717
AAPL이 마지막으로 등장하는 날짜:  20230831


In [14]:
# 'rld_ose_iem_tck_cd' 열에 있는 티커코드를 ','를 기준으로 분리하여 리스트로 만듦
df['rld_ose_iem_tck_cd_lst'] = df['rld_ose_iem_tck_cd'].str.split(',')

In [15]:
print(min(df[df['rld_ose_iem_tck_cd_lst'].apply(lambda x: 'AAPL' in x)]['rgs_dt']))
print(max(df[df['rld_ose_iem_tck_cd_lst'].apply(lambda x: 'AAPL' in x)]['rgs_dt']))
print(len(df[df['rld_ose_iem_tck_cd_lst'].apply(lambda x: 'AAPL' in x)]['rgs_dt']))

20230111
20230831
937


In [16]:
df[['rld_ose_iem_tck_cd', 'rld_ose_iem_tck_cd_lst']].head(10)

,rld_ose_iem_tck_cd,rld_ose_iem_tck_cd_lst
0,"CERT,CTMX","[CERT, CTMX]"
1,"TSLX,OCN","[TSLX, OCN]"
2,"PODD,DKNG,CERT,BITF","[PODD, DKNG, CERT, BITF]"
3,TMCI,[TMCI]
4,"NEO,WING","[NEO, WING]"
5,SLNA,[SLNA]
6,"OBT,SBNY","[OBT, SBNY]"
7,"NCLH,ISPO","[NCLH, ISPO]"
8,"NSA,DRH","[NSA, DRH]"
9,"AVDL,AVDL,RMTI,TGTX","[AVDL, AVDL, RMTI, TGTX]"


In [17]:
#'tck_iem_cd'와 'rld_ose_iem_tck_cd_lst'에 있는 모든 주식코드를 all_tck_iem_cd 열에 합치고, 중복된 티커코드가 여러 번 나타나는 경우 제거함

df['all_tck_iem_cd'] = df.apply(lambda row: list(set([row['tck_iem_cd']] + row['rld_ose_iem_tck_cd_lst'])), axis=1)

In [20]:
df[['tck_iem_cd', 'rld_ose_iem_tck_cd', 'all_tck_iem_cd']]

,tck_iem_cd,rld_ose_iem_tck_cd,all_tck_iem_cd
0,CERT,"CERT,CTMX","[CERT, CTMX]"
1,OCN,"TSLX,OCN","[OCN, TSLX]"
2,CERT,"PODD,DKNG,CERT,BITF","[CERT, DKNG, BITF, PODD]"
3,TMCI,TMCI,[TMCI]
4,NEO,"NEO,WING","[WING, NEO]"
...,...,...,...
2295444,ABEV,"KDP,KDP,ABEV","[ABEV, KDP]"
2295446,FFIV,FFIV,[FFIV]
2295450,EDU,EDU,[EDU]
2295452,TDY,TDY,[TDY]


In [21]:
df.drop('rld_ose_iem_tck_cd_lst', axis=1, inplace=True)

In [22]:
df.columns

Index(['rgs_dt', 'tck_iem_cd', 'til_ifo', 'ctgy_cfc_ifo', 'mdi_ifo',
       'news_smy_ifo', 'rld_ose_iem_tck_cd', 'url_ifo', 'all_tck_iem_cd'],
      dtype='object')

In [23]:
df.shape

(88602, 9)

In [24]:
df.to_csv(os.path.join(PATH, 'nasdaq_df_wo_text.csv'), index=False)

## Text Crawling
- 제공받은 nasdaq_rss_ifo의 url을 newspaper 라이브러리를 이용해 기사본문을 크롤링함.
- nasdaq_final.csv

In [27]:
!pip install newspaper3k

In [28]:
from newspaper import Article
import numpy as np
import pandas as pd
import time
import os
import re

In [29]:
nasdaq_df = pd.read_csv(os.path.join(PATH, 'nasdaq_df_wo_text.csv'))
nasdaq_df.shape

(88602, 9)

In [30]:
def extract_text(url):

    article = Article(url)
    article.download()
    article.parse()
    text = article.text or 'N/A' #text가 없는 경우 'N/A'로 출력

    return text

In [32]:
nasdaq_df['text'] = None  # 'text'열을 None으로 초기화

In [34]:
nasdaq_df['text'].isna().sum()

88602

In [37]:
#크롤링 코드

from IPython.display import display

def crawl_nasdaq_texts(nasdaq_df, start_index, end_index): #start_index, end_index를 조정하여 여러 번에 나눠서 크롤링할 수 있음
    for index, row in nasdaq_df.iterrows():
        if index < start_index:
            continue

        try:
            url = row['url_ifo']
            extracted_text = extract_text(url)
            nasdaq_df.at[index, 'text'] = extracted_text

        #에러 나는 경우 처리(공백은 try 경우에 포함)
        except Exception as e:
            print(f"Error at index {index}: {e}")

        if index % 100 == 0:
            print(f"Processing index {index}")
            display(nasdaq_df.iloc[index-10:index])

        #아래 코드는 ckpt를 사용할 경우 주석 제거하시면 됩니다.

        # if index % 1000 == 0:
        #     folder_path = f'{PATH}'"/nasdaq_text_crawling_ckpt"
        #     if not os.path.exists(folder_path):
        #         os.makedirs(folder_path)
        #     nasdaq_df.to_csv(os.path.join(PATH, "nasdaq_text_crawling_ckpt", f"{start_index}_{index}_ckpt.csv"), index=False)

        #index > end_index인 경우 자동으로 제거
        if index == end_index:
            folder_path = f'{PATH}'"/nasdaq_text_crawling_ckpt"
            if not os.path.exists(folder_path):
                os.makedirs(folder_path)
            nasdaq_df.to_csv(os.path.join(PATH, "nasdaq_text_crawling_ckpt", f"{start_index}_{index}_ckpt.csv"), index=False)
            print(f"{start_index}_{index} 크롤링 완료")
            break

    return nasdaq_df

In [ ]:
result_df = crawl_nasdaq_texts(nasdaq_df, 0, len(nasdaq_df)-1)

In [ ]:
#중복 행 제거
result_df.drop_duplicates(inplace=True)
result_df.shape

In [ ]:
# 'text'로 시작하는 모든 열을 필터링
text_columns = [col for col in result_df.columns if col.startswith('text')]
print(text_columns)

# 해당 열들에서 'NaN'(에러가 난 행)이 아닌 첫 번째 값을 찾는 새로운 열 생성. 모두 'NaN'일 경우, 'NaN'으로 저장
result_df['text_not_nan'] = result_df[text_columns].apply(lambda row: next((item for item in row if not pd.isna(item)), np.nan), axis=1)

result_df.head()

In [ ]:
result_df = result_df[result_df['text_not_nan']!='N/A'] #text_not_nan이 'N/A'가 아닌 행만 필터링(text가 공백인 행)

In [ ]:
result_df.isna().sum() #text_not_nan에 null값(error난 행) 있는지 확인.

In [ ]:
result_df.drop(text_columns, axis=1, inplace=True) #바로 위에서 정의한 text로 시작하는 열 drop(text_not_nan은 drop 안 됨)

result_df.rename(columns={'text_not_nan': 'text'}, inplace=True)

result_df.head()

In [ ]:
#error난 url 다시 크롤링 시도한 후 최종 결과를 nasdaq_final.csv에 저장

nan_index_range = result_df[result_df['text'].isna()].index
print("NaN indices: ", nan_index_range)

def recrawl_nasdaq_error_texts(nasdaq_df, start_index=0, end_index=len(df)-1, nan_index_range=None): #원하는 index range에서 nan_index_range 찾을 수 있음
    error_url_list = []
    for index in nan_index_range:
        if index < start_index:
            continue

        # #end_index를 작게 설정하는 경우 대비. end_index=len(df)-1이라면 필요 없음.
        # if index > end_index:
        #     nasdaq_df.to_csv(os.path.join(PATH, "nasdaq_final.csv"), index=False)
        #     print(f"{start_index}_{end_index} 크롤링 완료")
        #     break


        try:
            url = nasdaq_df.at[index, 'url_ifo']
            extracted_text = extract_text(url)
            nasdaq_df.at[index, 'text'] = extracted_text
            print(f"Processing index {index}")

        except Exception as e:
            print(f"Error at index {index}: {e}")
            error_url_list.append(url)

    #error가 난 url이 있는 행 포함하여 저장
    print("Number of error urls: ", len(error_url_list))
    nasdaq_df.to_csv(os.path.join(PATH, "nasdaq_final.csv"), index=False)

    return nasdaq_df, error_url_list

In [ ]:
df, error_url_list = recrawl_nasdaq_error_texts(result_df, nan_index_range=nan_index_range)

# 2. Stock Description Crawling(현재 작동하지 않는 코드)
- 야후파이낸스의 라이브러리를 이용해 기업 설명이 기재된 description을 크롤링함.
- 추후 토픽의 키워드와 키워드에 맞는 기업을 연결하기 위함.
- stock_description.csv

In [39]:
import yfinance as yf
import pandas as pd
import warnings
import os
warnings.filterwarnings("ignore")

In [40]:
PATH

'NHIS_BDC_2023/Round1/'

In [41]:
stock = pd.read_csv(os.path.join(PATH, 'NASDAQ_FC_STK_IEM_IFO.csv'), encoding = "cp949")
stock

,isin_cd,tck_iem_cd,fc_sec_krl_nm,fc_sec_eng_nm
0,US00211V1061,AACG,ATA ...,ATA CreatGlo ...
1,US00032Q1040,AADI,Aadi Bioscience ...,Aadi Bioscience ...
2,US02376R1023,AAL,아메리칸 에어라인스 그룹 ...,American Airline ...
3,US03823U1025,AAOI,어플라이드 옵토일렉트로닉스 ...,AOI ...
4,US0003602069,AAON,에이에이온 ...,AAON ...
...,...,...,...,...
2738,US4884452065,ZVRA,Zevra ...,Zevra ...
2739,US98987D1028,ZVSA,Zyversa ...,Zyversa ...
2740,US98985Y1082,ZYME,Zymeworks ...,Zymeworks ...
2741,US98986X1090,ZYNE,자이너바 파마수티컬스 ...,Zynerba Pharms ...


In [42]:
# 'tck_iem_cd' 열의 공백 제거
stock['tck_iem_cd'] = stock['tck_iem_cd'].str.strip()

In [43]:
# 종목 티커 코드 리스트
tck_iem_cds = list(stock['tck_iem_cd'])

In [44]:
# 종목 description 추출
def get_stock_descriptions(tck_iem_cds):
    descriptions = []

    for tck_iem_cd in tck_iem_cds:
        try:
            # Ticker 객체 생성
            ticker = yf.Ticker(tck_iem_cd)

            # 종목 정보 가져오기 - 예선 때는 작동했으나 현재 작동하지 않는 코드입니다
            stock_info = ticker.info 

            # 'longBusinessSummary' 키에 해당하는 종목 description 정보 가져오기
            description = stock_info.get('longBusinessSummary', None)

            # 가져온 정보를 리스트에 추가
            descriptions.append(description)
        except Exception as e:
            print(f"Error fetching data for {tck_iem_cd}: {str(e)}")
            descriptions.append(None)

    data = pd.DataFrame({'tck_iem_cd': tck_iem_cds, 'description': descriptions})

    return data

In [ ]:
#데이터프레임 생성
stock_description = get_stock_descriptions(tck_iem_cds)

stock_description

In [ ]:
#결측치 확인
stock_description.isnull().sum()

In [ ]:
#전처리
stock_description = stock_description.applymap(lambda x: x.strip() if isinstance(x, str) else x)

In [ ]:
stock_description.dropna(subset=['description'], inplace=True)
stock_description.reset_index(drop=True, inplace=True)
stock_description.drop_duplicates(subset=['description'], inplace=True)

In [ ]:
stock_description.head()

In [ ]:
stock_description.shape

### 데이터프레임에 영문기업명 추가
- 추후 FinBERT로 감성분석 진행시 영문기업명을 기준으로 기사 문단을 추출하기 위함.

In [ ]:
company_name = stock[['tck_iem_cd', 'fc_sec_eng_nm']]
company_name

In [ ]:
# 두 데이터프레임을 tck_iem_cd 열을 기준으로 inner join
stock_description_new = pd.merge(stock_description, company_name, how='inner', on='tck_iem_cd')
stock_description_new.head()

In [ ]:
stock_description_new.shape

In [ ]:
stock_description_new.to_csv(os.path.join(PATH, 'stock_description.csv'), index=False)

# 3. CNBC 기사 url 크롤링
- 크롤링 목적 : 토픽모델링 시에 사용할 데이터
- 데이터 선정 이유: CNBC는 미국의 경제·금융 뉴스 채널로, CNBC에서 각광받거나 언급량이 많은 토픽은 시장에서 많은 관심을 받고 있는 분야라고 여길 수 있다. 그렇기에 투자 기업을 정하기 전 CNBC를 통해 앞으로 성장 가능성이 있거나 시장의 개입이 이뤄질 것 같은 분야를 먼저 확인하고, 해당 분야에서 투자 가치가 있는 기업을 제공하고자 한다.

- 수집 데이터 일자 : 2023.1.1 ~ 2023.8.31

- 크롤링 방식 : Google에 "CNBC"를 검색하고, 7일 단위(2023.1.1 ~ 2023.1.7, ...)로 필터링을 주어, 해당 주의 모든 기사 url 크롤링

- 7일 단위로 필터링한 이유 : 구글 검색 결과의 제한(33페이지)으로 인해 기간을 길게 할 경우, 해당 기간의 기사가 모두 표시되지 않기 때문, 기간을 너무 짧게 할 경우에는 필요없는 기사가 추출됨을 확인했기에 7일이 적절하다고 판단.

- all_matching_links_final.pkl

- driver 문제로 <CNBC 기사 url 크롤링> 섹션에서는 가상환경에서 실행하셔야 합니다.
- conda create -n NH python=3.8
- selenium==4.12.0, pandas==2.0.3, re==2.2.1, bs4==4.12.2, requests==2.31.0, newspaper==0.2.8

In [48]:
!pip install selenium
!pip install bs4
!pip install requests
!pip install newspaper3k

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 72.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.2/400.2 kB 36.6 MB/s eta 0:00:00


In [49]:
import selenium
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
import re
from datetime import datetime, timedelta
import pickle
import os
from bs4 import BeautifulSoup
import requests
from newspaper import Article

In [50]:
PATH

'NHIS_BDC_2023/Round1/'

In [ ]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
driver = webdriver.Chrome(options=chrome_options)


def cnbc_crawling(start_date, end_date):

    global driver

    start_values_iter = iter(range(0, 320, 10)) #페이지 number - 0이면 10페이지
    current_matching_links = []
    retry_count = 0
    terminate_flag = False

    while True:
        if terminate_flag:
            break

        #구글 검색결과의 각 페이지에서 크롤링
        try:
            current_start = next(start_values_iter)
        except StopIteration:
            break

        print(f"page: {current_start // 10 + 1}")

        while True:
            matching_links_count = 0
            date_range_str = f"cd_min:{start_date},cd_max:{end_date}"
            google_search_url = f'https://www.google.com/search?q=cnbc&sca_esv=568517199&tbs=cdr:1,{date_range_str}&tbm=nws&sxsrf=AM9HkKmYdZ0_zRMPireMpaE6VsIgqW5jBg:1695740222154&ei=PvESZZmECbfh2roPnbOa-A0&start={current_start}&sa=N&ved=2ahUKEwiZm8rMxMiBAxW3sFYBHZ2ZBt84ygIQ8tMDegQIAxAW&biw=1137&bih=790&dpr=2'
            driver.get(google_search_url)

            links_with_class = driver.find_elements(By.CSS_SELECTOR, 'a.WlydOe[jsname="YKoRaf"]')
            for link in links_with_class:
                href = link.get_attribute('href')

                if href and re.match(r'https://www\.cnbc\.com/2023/\d{2}/\d{2}/[a-z0-9-]+\.html', href):
                    current_matching_links.append(href)
                    print(href)
                    matching_links_count += 1

            if matching_links_count > 0:
                retry_count = 0
                break

            #해당 페이지에 matching되는 url이 없으면 재시도 - "로봇이 아닙니다"가 뜨거나 페이지가 비었다면, matching되는 url이 없을 것
            else:
                driver.quit()
                driver = webdriver.Chrome(options=chrome_options)
                retry_count += 1

            #재시도 횟수가 2번 이상이면 terminate시키고 다음 기간으로 넘어감. (그 페이지가 비었다는 뜻이므로). 재시도란, 결과가 나오지 않아 드라이버를 껐다 켜는 행위.
            #"로봇이 아닙니다"는 driver를 새로 열면 사라지므로, 재시도 횟수가 2번 이상이라는 것은 정말 페이지가 비었다는 의미.
            if retry_count >= 2:
                terminate_flag = True
                break

    return current_matching_links

#함수 호출 예시 - 7일 단위로 날짜 설정해주기
cnbc_crawling("01/01/2023", "01/07/2023")

In [ ]:
#7일 단위로 크롤링

start_date = datetime.strptime("1/1/2023", "%m/%d/%Y")
end_date = datetime.strptime("8/31/2023", "%m/%d/%Y")
all_matching_links = [] #모든 link 저장
empty_count = 0

# Driver options
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
driver = webdriver.Chrome(options=chrome_options)

# 7일 범위 날짜 생성
while start_date <= end_date:
    next_end_date = start_date + timedelta(days=6)
    if next_end_date > end_date:
        next_end_date = end_date

    #7일 단위로 크롤링
    current_start_date, current_end_date = (start_date.strftime("%m/%d/%Y"), next_end_date.strftime("%m/%d/%Y"))
    current_matching_links = cnbc_crawling(current_start_date, current_end_date)
    all_matching_links += current_matching_links

    # 결과 처리 또는 저장
    print(f'크롤링 결과 (시작 날짜: {current_start_date}, 종료 날짜: {current_end_date}):')
    print(f'누적 크롤링된 기사 수: {len(all_matching_links)}')
    print(f'현재 크롤링된 기사 수: {len(current_matching_links)}')

    start_date += timedelta(days=7)


    # ###4주마다 체크포인트 저장
    # week_counter += 1  # Increment week_counter
    # if week_counter == 4:  # Check if 4 weeks have passed
    #     file_name = f"all_matching_links_{next_end_date.strftime('%Y_%m_%d')}.pkl"
    #     full_path = os.path.join(PATH, file_name)
    #     with open(full_path, 'wb') as f:
    #         pickle.dump(all_matching_links, f)
    #     week_counter = 0  # Reset week_counter

    # Save the remaining data if next_end_date reaches end_date

if next_end_date == end_date:
    file_name = "all_matching_links_final.pkl"
    full_path = os.path.join(PATH, file_name)
    with open(full_path, 'wb') as f:
            pickle.dump(all_matching_links, f)

## url에 접속하여 기사 정보 크롤링하기

- url에 접속하여, 제목, 날짜, 카테고리, Key Points(기사요약), 본문 크롤링
- 카테고리 크롤링 이유 : 분석 시, 주식 정보에 관련 없는 카테고리는 삭제하기 위함.
- Key Points 크롤링 이유 : 기사 요약본으로 토픽모델링을 진행하고자 했으나, 문자열이 짧아 토픽이 잘 추출되지 않았음.
- 본문 크롤링 이유 : 토픽모델링에 실제로 사용한 데이터, 본문 데이터로 토픽모델링을 진행한 결과 토픽이 가장 잘 추출되었음.
- cnbc_newsdata_final.pkl

In [ ]:
all_matching_links = pd.read_pickle(os.path.join(PATH, "all_matching_links_final.pkl"))

CNBC_CKPT_PATH = os.path.join(PATH, "news_info_ckpt")
if not os.path.exists(CNBC_CKPT_PATH):
    os.makedirs(CNBC_CKPT_PATH, exist_ok=True)


#Key Points 크롤링
def extract_key_points(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        target_div = soup.find('div', {'class': 'group'})
        if target_div:
            ul = target_div.find('ul')
            if ul:
                lis = ul.find_all('li')
                return ' '.join([li.text for li in lis])
    except Exception as e:
        print(f"Error in extract_key_points: {e}")
        return ""


chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
driver = webdriver.Chrome(options=chrome_options)

# 기사의 정보를 저장할 list 생성
articles_data = []


def extract_article_info(url, retry_count=0): #재시도 횟수 0에서 시작
    global driver  # 드라이버를 글로벌 변수로 설정

    #category 크롤링
    try:
        driver.get(url)
        page_html = driver.page_source
        soup = BeautifulSoup(page_html, 'html.parser')

        article_header = soup.find(class_='ArticleHeader-eyebrow') or soup.find(class_='ArticleHeader-styles-makeit-eyebrow--Degp4')

        category = article_header.text if article_header else "N/A"
    except Exception as e:
        print(f"Error in category extraction: {e}")
        category = ""


    #title, date, key_points, text 크롤링
    try:
        article = Article(url)
        article.download()
        article.parse()
        title = article.title or 'N/A'
        date = article.publish_date or 'N/A'
        text = article.text or 'N/A'
        key_points = extract_key_points(url) or 'N/A'


    except Exception as e:
        print(f"Error in newspaper extraction: {e}")

        # 에러가 발생하면 드라이버 재시작
        driver.quit()
        chrome_options = webdriver.ChromeOptions()
        chrome_options.add_argument('--headless')
        driver = webdriver.Chrome(options=chrome_options)

        # retry_count < 1이면 크롤링 다시 시도
        if retry_count < 1:
            return extract_article_info(url, retry_count=retry_count + 1) #재귀적 코드

        #retry_count >= 1일 때, url을 제외한 행 전체가 빈칸으로 나타남

        print(f'Error in {url}')
        title, date, text, key_points = "", "", "", ""

    new_data = {
        'title': title,
        'date': date,
        'category': category,
        'key_points': key_points,
        'text': text,
        'url': url
    }

    #새로 크롤링한 데이터를 articles_data list에 추가
    articles_data.append(new_data)
    #print(f"Newly appended data: {new_data}")

#url list 정의 - link를 날짜 순서대로 정렬
url_list = sorted(list(all_matching_links))

for idx, url in enumerate(url_list, 1):
    extract_article_info(url)

    #10개 단위로 체크포인트 파일 저장
    if idx % 10 == 0:
        checkpoint_df = pd.DataFrame(articles_data)
        display(checkpoint_df)

        checkpoint_df.to_csv(os.path.join(CNBC_CKPT_PATH, f"{idx}번째_체크포인트.csv"), index=False)
        print(f"Saved checkpoint at {idx}th URL.")

    #time.sleep(1.5)

driver.quit()

df = pd.DataFrame(articles_data)

df

In [ ]:
df[(df=='').any(axis=1)]

In [ ]:
df_filtered = df[df['title'] != '']

df_filtered

In [ ]:
df_filtered.to_csv(os.path.join(PATH, "cnbc_newsdata_final.pkl", index=False))

In [ ]:
# with open(os.path.join(PATH, "cnbc_newsdata_final.pkl"), 'wb') as f:
#     pickle.dump(df_filtered, f)